In [184]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [185]:
df = pd.read_pickle("data.pkl")

In [186]:
df.head()

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,1,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,3,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,4,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,5,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [187]:
def year_to_month(s):
    # Converts input of format "xx years yy months" or "xx years" into a month integer
    splitted = s.split()
    # Data is in format "xx years yy months" or "xx years"
    months = 0
    if len(splitted) >= 4:  # Handle the format "xx years yy months" 
        months += int(splitted[0]) * 12 # 
        months += int(splitted[2])
    else:
        months += int(splitted[0]) * 12
        
    return months

def storey_range(s):
    # converts input of format "xx to xx" to the average of the 2 numbers.
    splitted = s.split()
    val1 = int(splitted[0])
    val2 = int(splitted[2])
    avg_val = (val1 + val2) / 2
    return avg_val
    
def flat_type_to_integer(s):
    # Map flat_type to integer to reduce dimensions when get_dummies.
    mapping = {"1 ROOM":1, "2 ROOM":2, "3 ROOM":3,
               "4 ROOM":4, "5 ROOM": 5, "EXECUTIVE":6, "MULTI-GENERATION":7}
    if s not in mapping:
        return 0
    return mapping[s]


def getdummy_inplace(local_df):
    local_df = pd.concat([local_df, 
                            pd.get_dummies(local_df.select_dtypes(exclude=['number', 'datetime']))], axis=1)
    cols_to_drop = local_df.select_dtypes(exclude=['number', 'datetime']).columns
    local_df.drop(columns=cols_to_drop, inplace=True)
    return local_df
    

    

In [188]:
# _id: set it as index. 
df.set_index("_id", inplace=True)

# month: change this to a float, representing year of transacation.
df['month'] = pd.to_datetime(df['month'])
df['month'] = df['month'].dt.year
df.rename(columns={'month':'year_of_sale'}, inplace=True)

# street_name: Drop if we have town...
df.drop(columns="street_name", inplace=True)

# storey range: Take middle of the storey range
df['storey_range'] = df['storey_range'].apply(storey_range)

#lease commence date: Can drop if we have remaining lease?
df.drop(columns="lease_commence_date", inplace=True)

# flat model: drop
df.drop(columns="flat_model", inplace=True)

# remaining lease: convert to months (integer)
df['remaining_lease'] = df['remaining_lease'].apply(year_to_month)

# block: does it contain any useful information? No.
df.drop(columns="block",inplace=True)

# flat type: map to integers.
df['flat_type'] = df['flat_type'].apply(flat_type_to_integer)

# get dummy cols. 
df = getdummy_inplace(df)

# Bring resale_price to last col
df.insert(len(df.columns), 'resale_price_tmp', df['resale_price'])
df.drop(columns='resale_price',inplace=True)
df.rename(columns={'resale_price_tmp':'resale_price'})

,year_of_sale,flat_type,storey_range,floor_area_sqm,remaining_lease,town_ANG MO KIO,town_BEDOK,town_BISHAN,town_BUKIT BATOK,town_BUKIT MERAH,...,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN,resale_price
_id,,,,,,,,,,,,,,,,,,,,,
1,2017,2,11.0,44.0,736,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,232000.0
2,2017,3,2.0,67.0,727,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,250000.0
3,2017,3,2.0,67.0,749,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,262000.0
4,2017,3,5.0,68.0,745,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,265000.0
5,2017,3,2.0,67.0,749,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,265000.0
6,2017,3,2.0,68.0,756,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,275000.0
7,2017,3,5.0,68.0,738,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,280000.0
8,2017,3,5.0,67.0,700,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,285000.0
9,2017,3,5.0,68.0,738,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,285000.0


In [189]:
df.head()

,year_of_sale,flat_type,storey_range,floor_area_sqm,remaining_lease,town_ANG MO KIO,town_BEDOK,town_BISHAN,town_BUKIT BATOK,town_BUKIT MERAH,...,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN,resale_price_tmp
_id,,,,,,,,,,,,,,,,,,,,,
1,2017,2,11.0,44.0,736,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,232000.0
2,2017,3,2.0,67.0,727,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,250000.0
3,2017,3,2.0,67.0,749,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,262000.0
4,2017,3,5.0,68.0,745,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,265000.0
5,2017,3,2.0,67.0,749,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,265000.0


In [190]:
df.head()

,year_of_sale,flat_type,storey_range,floor_area_sqm,remaining_lease,town_ANG MO KIO,town_BEDOK,town_BISHAN,town_BUKIT BATOK,town_BUKIT MERAH,...,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN,resale_price_tmp
_id,,,,,,,,,,,,,,,,,,,,,
1,2017,2,11.0,44.0,736,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,232000.0
2,2017,3,2.0,67.0,727,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,250000.0
3,2017,3,2.0,67.0,749,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,262000.0
4,2017,3,5.0,68.0,745,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,265000.0
5,2017,3,2.0,67.0,749,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,265000.0


In [196]:
# Start train-test split and pipeline


rfr = RandomForestRegressor()

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)


pipe = Pipeline([('scalar', StandardScaler()),('rfr',rfr)])
pipe.fit(X_train, y_train)

/Users/james/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/james/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/james/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('scalar', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rfr', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [197]:
pipe.score(X_test, y_test)

/Users/james/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.9325470649659051

In [195]:
# standardize numeric cols
y

_id
1        232000.0
2        250000.0
3        262000.0
4        265000.0
5        265000.0
6        275000.0
7        280000.0
8        285000.0
9        285000.0
10       285000.0
11       288500.0
12       295000.0
13       295000.0
14       297000.0
15       298000.0
16       298000.0
17       300000.0
18       301000.0
19       306000.0
20       312000.0
21       312000.0
22       321000.0
23       325000.0
24       325000.0
25       328000.0
26       330000.0
27       330000.0
28       335000.0
29       338000.0
30       366000.0
           ...   
68132    410000.0
68133    510000.0
68134    530000.0
68135    530888.0
68136    493000.0
68137    428000.0
68138    412000.0
68139    405000.0
68140    400000.0
68141    405000.0
68142    510000.0
68143    457000.0
68144    458000.0
68145    418888.0
68146    383000.0
68147    510000.0
68148    515000.0
68149    460000.0
68150    522000.0
68151    515000.0
68152    510000.0
68153    533000.0
68154    448000.0
68155    490000.0
68156 